In [1]:
import chess
import chess.pgn
import chess.polyglot
import os
import sys
import time
import pickle
import numpy as np
import pandas as pd
from datetime import datetime

data_path = r'e:\projects\Python\FindingElo\data\Lichess\\'
data_path2 = r'f:\Lichess\\'

In [2]:
def parse(games, path, name):
    # Создаем недостающие папки
    if not os.path.isdir(path):
        os.mkdir(path)
        os.mkdir(path+'pgn')
        os.mkdir(path+'pickle')
    else:
        if not os.path.isdir(path+'pgn'):
            os.mkdir(path+'pgn')
        if not os.path.isdir(path+'pickle'):
            os.mkdir(path+'pickle')
    # Открываем файл для записи "отсеяных" pgn
    new_pgn = open(path+'pgn\\'+name, "w", encoding="utf-8")
    exporter = chess.pgn.FileExporter(new_pgn)
    # Создаем массивы под необходимые данные
    res = []
    w_elos = []
    b_elos = []
    utc_date = []
    eco = []
    opening = []
    time_control = []
    termination = []
    for g in games:
        w_elos.append(g.headers['WhiteElo'])
        b_elos.append(g.headers['BlackElo'])
        utc_date.append(datetime.strptime(g.headers['UTCDate'], '%Y.%m.%d'))
        eco.append(g.headers['ECO'])
        opening.append(g.headers['Opening'])
        time_control.append(g.headers['TimeControl'])
        termination.append(g.headers['Termination'])
        r = g.headers['Result']
        if r == '1/2-1/2':
            res.append(0.5)
        elif r == '1-0':
            res.append(1)
        else:
            res.append(0)
        g.accept(exporter)
    # Преобразуем полученные данные в датафреймы и объединяем их
    elos = pd.DataFrame(np.vstack((w_elos, b_elos)).T,columns=['ELO_white','ELO_black'])
    utc_date = pd.DataFrame(utc_date, columns=['UTCDate'])
    eco = pd.DataFrame(eco, columns=['ECO'])
    opening = pd.DataFrame(opening, columns=['Opening'])
    time_control = pd.DataFrame(time_control, columns=['TimeControl'])
    termination = pd.DataFrame(termination, columns=['Termination'])
    results = pd.DataFrame(res, columns=['Result'])
    data = elos.join(results, how='outer')\
        .join(eco).join(opening)\
        .join(utc_date).join(time_control)\
        .join(termination)#.fillna(method='pad')
    # Сохраняем результат
    data.to_pickle(path+'pickle\\'+name+'.pickle')
    return data

In [3]:
def load_games(path_to_file, name, n_games=sys.maxsize):
    # если существуют сохраненные файлы, то загружаем данные из них
    main_dir = 'stockfish\\'
    if os.path.exists(path_to_file+main_dir+'pickle\\'+name+'.pickle'):
        with open(path_to_file+main_dir+'pickle\\'+name+'.pickle', 'rb') as f:
            parsed_data = pd.read_pickle(f)
        return parsed_data
    # если файлов нет, то выполняем обработку данных
    # Для начала считываем данные о играх
    with open(path_to_file+name) as data:
        game = chess.pgn.read_game(data)
        games = []
        count = 0
        while game is not None and count<n_games:
            if game.variations != [] and game.variation(0).eval() is not None:
                games.append(game)
            game = chess.pgn.read_game(data)
            count += 1
    # Далее обрабатываем данные
    parsed_data = parse(games, path_to_file+main_dir, name)
    return parsed_data

In [67]:
lichess_pgns = []
for f_name in os.listdir(data_path):
    if f_name.endswith('.pgn'):
        lichess_pgns.append(f_name)
lichess_pgns

['lichess_db_standard_rated_2013-01.pgn',
 'lichess_db_standard_rated_2013-02.pgn',
 'lichess_db_standard_rated_2013-03.pgn',
 'lichess_db_standard_rated_2013-04.pgn',
 'lichess_db_standard_rated_2013-05.pgn',
 'lichess_db_standard_rated_2013-06.pgn',
 'lichess_db_standard_rated_2013-07.pgn',
 'lichess_db_standard_rated_2013-08.pgn',
 'lichess_db_standard_rated_2013-09.pgn',
 'lichess_db_standard_rated_2013-10.pgn',
 'lichess_db_standard_rated_2013-11.pgn',
 'lichess_db_standard_rated_2013-12.pgn',
 'lichess_db_standard_rated_2014-01.pgn',
 'lichess_db_standard_rated_2014-02.pgn',
 'lichess_db_standard_rated_2014-03.pgn',
 'lichess_db_standard_rated_2014-04.pgn',
 'lichess_db_standard_rated_2014-05.pgn',
 'lichess_db_standard_rated_2014-06.pgn',
 'lichess_db_standard_rated_2014-07.pgn']

In [68]:
for name in lichess_pgns:
    start = time.time()
    _ = load_games(data_path, name)
    print(time.time()-start)

0.026948928833007812
0.0009458065032958984
0.001996278762817383
0.0010368824005126953
0.001985788345336914
0.0010030269622802734
806.6256296634674
886.8841643333435
876.592226266861
1205.6841132640839
1500.8996183872223
1672.1017923355103
2027.7437875270844
2018.20672082901


error during pgn parsing
Traceback (most recent call last):
  File "e:\programs\python3_10\lib\site-packages\chess\__init__.py", line 2960, in parse_san
    return next(move for move in self.generate_castling_moves() if self.is_queenside_castling(move))
StopIteration

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "e:\programs\python3_10\lib\site-packages\chess\pgn.py", line 1667, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "e:\programs\python3_10\lib\site-packages\chess\pgn.py", line 1045, in parse_san
    return board.parse_san(san)
  File "e:\programs\python3_10\lib\site-packages\chess\__init__.py", line 2962, in parse_san
    raise ValueError(f"illegal san: {san!r} in {self.fen()}")
ValueError: illegal san: 'O-O-O' in r3k3/ppp1nr2/1qn5/6P1/4P2Q/2P5/PP4P1/RN1RKBb1 w Qq - 1 22


2367.910896539688
2536.5791289806366
3299.2773265838623
4038.107279777527
0.4652106761932373


In [9]:
def filter_pickle(read_path, name, write_path=data_path, n_games=sys.maxsize):
    write_path += 'stockfish\\top_pickle'
    # создаем недостающие папки
    if not os.path.isdir(write_path):
        os.mkdir(write_path)
    # Проверяем, есть ли уже "обработанный" файл
    if os.path.exists(write_path+'\\'+name):
        with open(write_path+'\\'+name, 'rb') as f:
            parsed_data = pd.read_pickle(f)
        return parsed_data
    with open(read_path+name, 'rb') as f:
        # Считываем датафрейм
        parsed_data = pd.read_pickle(f)
        # Сохраняем индексы, по ним будет оставлять игры из pgn-файлов
        parsed_data['old_index'] = parsed_data.index
        # Удаляем данные с пропущенными значениями
        parsed_data.TimeControl = parsed_data.TimeControl.replace('-', None)
        parsed_data = parsed_data.replace('?', None).dropna()
        # Преобразуем ELO к числовому типу
        parsed_data.ELO_white = parsed_data.ELO_white.astype('float')
        parsed_data.ELO_black = parsed_data.ELO_black.astype('float')
        # Оставляем только партии с рейтингом выше 2400
        parsed_data = parsed_data[(parsed_data.ELO_white>2400) | (parsed_data.ELO_black>2400)].reset_index()
        # Категоризуем время партий
        for i in range(parsed_data.shape[0]):
            ti = int(parsed_data.loc[i,'TimeControl'].split('+')[0])
            if ti<180:
                parsed_data.loc[i,'TimeControl'] = 'bullet'
            elif ti<600:
                parsed_data.loc[i,'TimeControl'] = 'blitz'
            elif ti<1800:
                parsed_data.loc[i,'TimeControl'] = 'rapid'
            else:
                parsed_data.loc[i,'TimeControl'] = 'classical'
        parsed_data = parsed_data.drop('index', axis=1)
        # Сохраняем результат
        parsed_data.to_pickle(write_path+'\\'+name)
        return parsed_data

In [10]:
filtered_pickle = []
for f_name in os.listdir(data_path+'stockfish\\pickle\\'):
    if f_name.endswith('.pickle'):
        filtered_pickle.append(f_name)
filtered_pickle

['lichess_db_standard_rated_2013-01.pgn.pickle',
 'lichess_db_standard_rated_2013-02.pgn.pickle',
 'lichess_db_standard_rated_2013-03.pgn.pickle',
 'lichess_db_standard_rated_2013-04.pgn.pickle',
 'lichess_db_standard_rated_2013-05.pgn.pickle',
 'lichess_db_standard_rated_2013-06.pgn.pickle',
 'lichess_db_standard_rated_2013-07.pgn.pickle',
 'lichess_db_standard_rated_2013-08.pgn.pickle',
 'lichess_db_standard_rated_2013-09.pgn.pickle',
 'lichess_db_standard_rated_2013-10.pgn.pickle',
 'lichess_db_standard_rated_2013-11.pgn.pickle',
 'lichess_db_standard_rated_2013-12.pgn.pickle',
 'lichess_db_standard_rated_2014-01.pgn.pickle',
 'lichess_db_standard_rated_2014-02.pgn.pickle',
 'lichess_db_standard_rated_2014-03.pgn.pickle',
 'lichess_db_standard_rated_2014-04.pgn.pickle',
 'lichess_db_standard_rated_2014-05.pgn.pickle',
 'lichess_db_standard_rated_2014-06.pgn.pickle',
 'lichess_db_standard_rated_2014-07.pgn.pickle',
 'lichess_db_standard_rated_2014-08.pgn.pickle',
 'lichess_db_standar

In [11]:
top_pickle = []
for name in filtered_pickle:
    start = time.time()
    top_pickle.append(filter_pickle(data_path+'stockfish\\pickle\\', name))
    print(time.time()-start)

0.27942562103271484
0.009970664978027344
0.026926279067993164
0.0019943714141845703
0.001995086669921875
0.015957355499267578
0.019947052001953125
0.026928424835205078
0.030918359756469727
0.05086159706115723
0.17716574668884277
0.2543201446533203
0.3645360469818115
0.4218723773956299
0.3869643211364746
0.34470391273498535
0.8014459609985352
1.000939130783081
0.000997781753540039
0.0010066032409667969
0.0019855499267578125
0.0009963512420654297
0.002991914749145508
0.0009965896606445312
1.7823374271392822
0.0009899139404296875
0.0009970664978027344
0.001994609832763672
0.0009968280792236328
0.0019953250885009766
0.0019941329956054688
0.002991914749145508


In [12]:
def filter_pgn(read_path, name, write_path=data_path2, n_games=sys.maxsize):
    write_path += 'stockfish\\top_pgn'
    # создаем недостающие папки
    if not os.path.isdir(write_path):
        os.mkdir(write_path)
    # Проверяем, есть ли уже "обработанный" файл
    if os.path.exists(write_path+'\\'+name):
        return
    # Открываем файл для записи "отсеяных" pgn
    new_pgn = open(write_path+'\\'+name, "w", encoding="utf-8")
    exporter = chess.pgn.FileExporter(new_pgn)
    with open(read_path+name) as f:
        # Считываем pgns
        game = chess.pgn.read_game(f)
        while game is not None:
            # Удаляем данные с пропущенными значениями
            if game.headers['TimeControl'] == '-' or game.headers['WhiteElo'] == '?' or\
                game.headers['BlackElo'] == '?':
                game = chess.pgn.read_game(f)
                continue
            # Оставляем только партии с рейтингом выше 2400
            if float(game.headers['WhiteElo'])>2400 or float(game.headers['BlackElo'])>2400:
                game.accept(exporter)
            game = chess.pgn.read_game(f)
        return

In [15]:
def filter_pgn2(read_path, name, indexes, write_path=data_path):
    write_path += 'stockfish\\top_pgn'
    # создаем недостающие папки
    if not os.path.isdir(write_path):
        os.mkdir(write_path)
    # Проверяем, есть ли уже "обработанный" файл
    if os.path.exists(write_path+'\\'+name):
        return
    # Открываем файл для записи "отсеяных" pgn
    new_pgn = open(write_path+'\\'+name, "w", encoding="utf-8")
    exporter = chess.pgn.FileExporter(new_pgn)
    try:
        with open(read_path+name) as f:
            # Считываем pgns
            game = chess.pgn.read_game(f)
            i = 0
            while game is not None:
                if i in indexes:
                    game.accept(exporter)
                i += 1
                game = chess.pgn.read_game(f)
    except:
        with open(read_path+name, encoding="utf-8") as f:
            # Считываем pgns
            game = chess.pgn.read_game(f)
            i = 0
            while game is not None:
                if i in indexes:
                    game.accept(exporter)
                i += 1
                game = chess.pgn.read_game(f)
    return

In [17]:
filtered_pgns = []
for f_name in os.listdir(data_path+'stockfish\\pgn\\'):
    if f_name.endswith('.pgn'):
        filtered_pgns.append(f_name)
filtered_pgns

['lichess_db_standard_rated_2013-01.pgn',
 'lichess_db_standard_rated_2013-02.pgn',
 'lichess_db_standard_rated_2013-03.pgn',
 'lichess_db_standard_rated_2013-04.pgn',
 'lichess_db_standard_rated_2013-05.pgn',
 'lichess_db_standard_rated_2013-06.pgn',
 'lichess_db_standard_rated_2013-07.pgn',
 'lichess_db_standard_rated_2013-08.pgn',
 'lichess_db_standard_rated_2013-09.pgn',
 'lichess_db_standard_rated_2013-10.pgn',
 'lichess_db_standard_rated_2013-11.pgn',
 'lichess_db_standard_rated_2013-12.pgn',
 'lichess_db_standard_rated_2014-01.pgn',
 'lichess_db_standard_rated_2014-02.pgn',
 'lichess_db_standard_rated_2014-03.pgn',
 'lichess_db_standard_rated_2014-04.pgn',
 'lichess_db_standard_rated_2014-05.pgn',
 'lichess_db_standard_rated_2014-06.pgn',
 'lichess_db_standard_rated_2014-07.pgn',
 'lichess_db_standard_rated_2014-08.pgn',
 'lichess_db_standard_rated_2014-09.pgn',
 'lichess_db_standard_rated_2014-10.pgn',
 'lichess_db_standard_rated_2014-11.pgn',
 'lichess_db_standard_rated_2015-0

In [18]:
'''for name in filtered_pgns:
    start = time.time()
    filter_pgn(data_path2+'stockfish\\pgn\\', name)
    print(time.time()-start)'''

"for name in filtered_pgns:\n    start = time.time()\n    filter_pgn(data_path2+'stockfish\\pgn\\', name)\n    print(time.time()-start)"

In [20]:
i = 0
for name in filtered_pgns:
    start = time.time()
    filter_pgn2(data_path+'stockfish\\pgn\\', name, list(top_pickle[i].old_index.values))
    i += 1
    print(time.time()-start)

0.0
0.0
0.0
0.0009987354278564453
0.0
0.0
0.0009963512420654297
8.19352650642395
8.190094947814941
50.565847635269165
98.35171437263489
133.10262060165405
154.2813856601715
132.87454676628113
150.93637657165527
209.55738019943237
476.53973269462585
597.2588858604431
0.002002239227294922
0.0
0.0
0.0010001659393310547
0.0
0.0
0.0009989738464355469
0.0020008087158203125
0.0
0.0009906291961669922
0.0009984970092773438
0.0009953975677490234
0.0010004043579101562
0.0009961128234863281


In [21]:
# Проверяю, чтобы в записанных ".pickle" были те же данные,
# что и в ".pgn".
top_pickle[-1].head()

,ELO_white,ELO_black,Result,ECO,Opening,UTCDate,TimeControl,Termination,old_index
0,2614.0,2575.0,1.0,A08,King's Indian Attack,2015-08-31,bullet,Time forfeit,64
1,2446.0,2007.0,1.0,D31,Semi-Slav Defense: Marshall Gambit,2015-08-31,blitz,Normal,67
2,2451.0,2004.0,1.0,D03,"Queen's Pawn Game: Torre Attack, Gossip Variation",2015-08-31,blitz,Time forfeit,174
3,1638.0,2567.0,0.0,B76,"Sicilian Defense: Dragon Variation, Yugoslav A...",2015-08-31,bullet,Normal,243
4,2567.0,1916.0,1.0,C14,"French Defense: Classical Variation, Steinitz ...",2015-08-31,bullet,Normal,272


In [26]:
with open(data_path+r'stockfish\top_pgn\lichess_db_standard_rated_2015-09.pgn') as f:
    # Считываем pgns
    game = chess.pgn.read_game(f)
    i = 0
    while game is not None and i < 5:
        i += 1
        print(game.headers['WhiteElo'],
        game.headers['BlackElo'],
        game.headers['ECO'],
        game.headers['TimeControl'])
        game = chess.pgn.read_game(f)

2614 2575 A08 60+0
2446 2007 D31 180+0
2451 2004 D03 180+0
1638 2567 B76 60+0
2567 1916 C14 60+0
